# 头脑风暴

一些尝试

In [1]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

In [2]:
EN_BERT_PATH = './data/bert-base-uncased'

## 1. Embedding 的稳定性

同一个词做两次 embedding，结果是否相同？

In [3]:
# 加载模型
tokenizer = BertTokenizer.from_pretrained(EN_BERT_PATH)
model = BertModel.from_pretrained(EN_BERT_PATH)

Some weights of the model checkpoint at ./data/bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 批量计算 embeddings
def get_embeddings(corpus):
    encoded_inputs = tokenizer(corpus,
                               padding='max_length',
                               truncation=True,
                               return_tensors='pt')

    with torch.no_grad():
        outputs = model(**encoded_inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        # embeddings = outputs.last_hidden_state[:, 0, :]

    return embeddings

In [5]:
e1 = get_embeddings('usa')
e2 = get_embeddings('usa')

all((e1 == e2).tolist())

True

## 2. Embedding 差值的意义

In [6]:
corpus = [
    'king',
    'queen',
    'male',
    'female',
    'man',
    'woman',
    'blue',
    'bad'
]
embeddings = get_embeddings(corpus)
c2e = {c: e for c, e in zip(corpus, embeddings)}

In [7]:
def squared_distances(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.sum((a - b) ** 2)

In [8]:
squared_distances(a=c2e['king'],
                  b=c2e['queen'])

12.46646

In [9]:
squared_distances(a=c2e['male'],
                  b=c2e['female'])

11.630802

In [10]:
squared_distances(a=c2e['man'],
                  b=c2e['woman'])

20.933697

In [11]:
squared_distances(a=c2e['blue'],
                  b=c2e['bad'])

46.82113